In [1]:
import numpy as np
import pandas as pd
from keras.preprocessing.text import *
import matplotlib.pyplot as plt
from tokenizer import tokenizer

%matplotlib inline

PATH = '/home/epita/sim/sentiment_analysis/data/'
DETAILED_TRN_PATH = f'{PATH}/2018-Valence-oc-En-train.csv'
DETAILED_VAL_PATH = f'{PATH}/2018-Valence-oc-En-dev.csv'

TRN_PATH = f'{PATH}/training.1600000.processed.noemoticon.csv'


Using TensorFlow backend.


In [4]:
text = 'The quick brown fox jumped over the lazy dog.'

words = set(text_to_word_sequence(text))
vocab_size = len(words)
print(vocab_size)

result = one_hot(text, round(vocab_size*1.3))
print(result)

8
[6, 2, 7, 1, 4, 4, 6, 6, 7]


In [6]:
T = tokenizer.TweetTokenizer()

T.tokenize(text)

['The', 'quick', 'brown', 'fox', 'jumped', 'over', 'the', 'lazy', 'dog', '.']

In [9]:
tweets = pd.read_csv(TRN_PATH, sep=='\t')

In [16]:
tweets.sample(5)

,ID,Tweet,Affect Dimension,Intensity Class
75,2017-En-30021,When you wake up from a dream laughing at some...,valence,3: very positive emotional state can be inferred
1165,2017-En-31387,Good morning! Welcome the new day into your li...,valence,3: very positive emotional state can be inferred
63,2017-En-20042,@danisnotonfire you made me cry and shake to t...,valence,-3: very negative emotional state can be inferred
659,2017-En-30135,@LindseySanford @simplymeasured this makes me ...,valence,2: moderately positive emotional state can be ...
668,2017-En-40740,the rappers who stayed true to the game is rich.,valence,1: slightly positive emotional state can be in...


In [ ]:
Sequential()
Embedding(len(word_index) + 1, EMBEDDING_PATH...?, weights=[embedding_matrix], input_length=MAX_SEQUENCE_LENGTH, trainable=False)
LSTM(32)
Dropout(0.2)
Dense(32, 'relu')
Dropout(0.2)
Dense(3, activation='softmax')

model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['acc'])

model.fit(...)

In [ ]:
# transfer:

model.layers.pop()
model.layers.pop()
Dense(150, 'relu')
Dense(64, 'relu')
Dense(7, 'softmax')
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics['accuracy'])

# télécharger 50 millions de tweets pour le word2vec

#metric pour le script du prof: pearson
#utiliser pearson au lieu d'accuracy

# l'année dernière le prof a eu 78% accuracy en 

In [ ]:
#produire un fichier avec les mêmes noms de cartégories sur un fichier de test façon kaggle